In [1]:
from surya.foundation import FoundationPredictor
from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor
from PIL import Image
import tqdm
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import random
import requests
from io import BytesIO
from PIL import Image
from bs4 import BeautifulSoup
from tqdm import tqdm

import numpy as np


/Users/maryamsaad/Documents/Agentic_OCR/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
foundation_predictor = FoundationPredictor(device='mps')
recognition_predictor = RecognitionPredictor(foundation_predictor)
detection_predictor = DetectionPredictor()


In [3]:
import os
from pdf2image import convert_from_path
from tqdm import tqdm

def ocr_pdf(pdf_paths, dpi=300):
    images = []

    # Case 1: single PDF file (string)
    if isinstance(pdf_paths, str):
        if os.path.isfile(pdf_paths):
            return convert_from_path(pdf_paths, dpi=dpi)
        else:
            raise ValueError("Single path provided, but file does not exist.")

    # Case 2: list of PDF files
    elif isinstance(pdf_paths, list):
        for pdf_path in tqdm(pdf_paths, desc="Converting PDFs"):
            if os.path.isfile(pdf_path) and pdf_path.lower().endswith(".pdf"):
                imgs = convert_from_path(pdf_path, dpi=dpi)
                images.extend(imgs)
            else:
                print(f"Skipping invalid PDF: {pdf_path}")

        return images

    else:
        raise TypeError("pdf_paths must be a string or a list of PDF paths")

pdf_list=['/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/10 - 19/12650350001 8.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/20 - 29/12641980001 4.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/30 - 39/12711260001 5.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/40 - 49/12638690001 15.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/50-60/12640070001 P4.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/60- 69/12641550001 4.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/70 - 79/12639710001 5.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/80 - 90/12638700001 5.pdf',
          '/Users/maryamsaad/Documents/layout_detection/ocr/testing_data/90 - 99/12638720001 6.pdf'
          ]

pdfs=ocr_pdf("/Users/maryamsaad/Documents/layout_detection/ocr/simplified.pdf")

In [4]:

import re

def remove_tashkeel(text):
    # Arabic diacritics unicode range
    tashkeel_pattern = re.compile(r'[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]')
    return tashkeel_pattern.sub('', text)
def normalize_arabic(text):
    """
    Normalizes Arabic characters and removes OCR artifacts.
    """
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("  ", " ", text)
    text = re.sub("[ًٌٍَُِّْ]", "", text)
    text = re.sub(r"[\|\)\(\:\-\;\\\/]+", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text)
    text = re.sub(r'[A-Za-z0-9]+', '', text)
    text=text.replace("[غير واضح]", "")
    text=text.replace("<>", "")
    text=text.replace(">", "")
    text=text.replace(">", "")
    text=text.replace("\n", " ")
    text=remove_tashkeel(text)
    return text.strip()
def html_to_text(html_str: str) -> str:    
    """
    Converts HTML content to plain text.
    """
    soup = BeautifulSoup(html_str, "html.parser")
    text = soup.get_text(separator=" ")
    return text




In [ ]:
from PIL import Image
import json
def OCR(img): 
    predictions = recognition_predictor([img], det_predictor=detection_predictor) 
    ocr_text = '' 
    for item in predictions: 
        text_lines = item.text_lines 
        for text_line in text_lines: 
            text = text_line.text 
            ocr_text += text + '\n' 
    return ocr_text.strip()


def run_pdf_ocr(pdf_path, limit=None):
    # Load page images from PDF
    pages = ocr_pdf(pdf_path, dpi=300)
    results = {}
    predictions = []

    for idx, img_path in tqdm(enumerate(pages, start=1), desc="Running OCR"):
        
        # optional: limit number of pages
        if limit and idx > limit:
            break
        
        # Load image
        img = img_path  # already a PIL Image

        # OCR prediction
        pred_text = OCR(img)
        pred_text = normalize_arabic(pred_text)
        pred_text = html_to_text(pred_text)

        # Save for overall metrics
        predictions.append(pred_text)


        results[idx] = {
            "image": img_path, 
            "pred": pred_text
        }

    return results

results=run_pdf_ocr("/Users/maryamsaad/Documents/layout_detection/ocr/simplified.pdf")
# results=run_pdf_ocr(pdf_list)


Detecting text regions: 100%|██████████| 3/3 [00:02<00:00,  1.18it/s]


AttributeError: 'LayoutDetector' object has no attribute 'get_predictor'